In [5]:
import pandas as pd
import numpy as np
import onnxruntime
from sklearn.metrics import accuracy_score

class OnnxModelWrapper:
    """The wrapper allows to use ONNX models in the same way as sklearn models"""

    def __init__(self, model_path):
        try:
            self._session = onnxruntime.InferenceSession(model_path)
        except:
            raise Exception("Model not found at the specified path: {}".format(model_path))

    def predict(self, X):
        results = self._run_model(X)
        return np.array(results[0])

    def _run_model(self, X):
        if isinstance(X, pd.DataFrame):
            return self._session.run(None, {'X': X.values.astype(np.float32)})
        else:
            return self._session.run(None, {'X': X.reshape(1, -1).astype(np.float32)})

In [6]:
DATASET_PATH = 'data\investigation_train_large_checked.csv' # original file from Brightspace, no changes
MODEL_PATH = 'model.onnx'

data = pd.read_csv(DATASET_PATH)
X = data.drop(columns=['Ja', 'Nee', 'checked'])
y = data['checked']

model = OnnxModelWrapper("model_2\model_2.onnx")

y_predicted = model.predict(X)

accuracy = accuracy_score(y, y_predicted)
print(f"accuracy: {accuracy:.3f}")

accuracy: 0.927
